In [ ]:
from transformers import pipeline

In [ ]:
model_id = "openai/whisper-small"
pipe = pipeline("automatic-speech-recognition", model=model_id)

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
!pip install beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup

In [ ]:
import json
import re

In [ ]:
def transcribe_speech(filepath):
    output = pipe(
        filepath,
        generate_kwargs={
            "task": "transcribe"
        },
        chunk_length_s=30,
        batch_size=8,
    )
    return output["text"]

def zero_shot_classification(text):

    transcribed_text = text

    url = 'https://www.producthunt.com/categories'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    h2_tags = soup.find_all('h2')
    h2_texts = [tag.text.strip() for tag in h2_tags]
    categories = h2_texts

    results = classifier(transcribed_text, candidate_labels=categories)
    label_score_pairs = sorted(zip(results['labels'], results['scores']), key=lambda x: x[1], reverse=True)[:3]
    data = {label: score for label, score in label_score_pairs}
    return data

def get_products(all_products):
    input_dict = json.loads(all_products.replace("'", '"'))
    formatted_category = re.sub(r'[ &]+', '-', list(input_dict.keys())[0]).lower()
    full_url = f"https://www.producthunt.com/categories/{formatted_category}"
    response = requests.get(full_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = [div.get_text().strip() for div in soup.select('div[class="text-18 font-semibold text-blue"]')]
    descriptions = [div.get_text().strip() for div in soup.select('div[class="text-14 sm:text-16 font-normal text-light-gray mb-6"]')]
    products = list(zip(titles, descriptions))[:5]

    html_content = ""
    for title, description in products:
            html_content += f'''
            <div style="border: 1px solid #ccc; border-radius: 10px; padding: 20px; margin-bottom: 10px;
                        background: linear-gradient(to right, #f9f9f9, #e8e8e8); color: #333;">
                <h2 style="text-decoration: none; color: #2c3e50;">{title}</h2>
                <p style="color: #34495e;">{description}</p>
            </div>
            '''
    return html_content

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
css = """
.container {
    height: 100%;
}
.scrollable-content {
        flex-grow: 1;
        overflow-y: auto; /* Make content within scrollable */
        padding: 10px;
}
"""

In [ ]:
with gr.Blocks(css=css) as demo:
    with gr.Row(elem_classes=["container"]):
        with gr.Column():
            audio_input = gr.Audio(sources="upload", type="filepath", label="Your audio file")
            process_button = gr.Button("Process")
            transcription_output = gr.TextArea(label="Transcribed text", lines=5, placeholder="Transcribed text will appear here...", interactive=True)
            classify_button = gr.Button("Classify Text")
            classification_output = gr.Textbox(label="Classification Results", placeholder="Classification results will appear here...")
            get_product_button = gr.Button("Get opportunities")

            process_button.click(transcribe_speech, inputs=audio_input, outputs=transcription_output)
            classify_button.click(zero_shot_classification, inputs=transcription_output, outputs=classification_output)

        with gr.Column(elem_classes=["scrollable-content"]):
            get_product_output = gr.HTML()
            get_product_button.click(get_products, inputs=classification_output, outputs=get_product_output)

    demo.launch()